## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-15-03-50-59 +0000,bbc,Trump targets longtime MAGA ally Marjorie Tayl...,https://www.bbc.com/news/articles/c9wv4dx05q5o...
1,2025-11-15-03-36-52 +0000,nyt,Officers were hurt in a crash while protecting...,https://www.nytimes.com/live/2025/11/14/us/tru...
2,2025-11-15-03-31-11 +0000,bbc,China urges citizens not to visit Japan over T...,https://www.bbc.com/news/articles/crklvx2n7rzo...
3,2025-11-15-03-15-00 +0000,wsj,What Will It Take to Get Retailers to Kill the...,https://www.wsj.com/business/retail/what-will-...
4,2025-11-15-03-09-48 +0000,bbc,Trump says he will take legal action against B...,https://www.bbc.com/news/articles/c891jp9j79do...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,49
50,epstein,15
280,new,12
23,will,11
27,he,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
33,2025-11-15-01-02-00 +0000,wsj,Trump said he would ask the Justice Department...,https://www.wsj.com/politics/policy/trump-call...,126
6,2025-11-15-03-01-00 +0000,wsj,President Trump said he is withdrawing his sup...,https://www.wsj.com/politics/policy/trump-with...,124
124,2025-11-14-21-07-00 +0000,wsj,Trump said he is withdrawing the nomination of...,https://www.wsj.com/politics/policy/trump-with...,102
200,2025-11-14-16-16-56 +0000,nypost,Goldman Sachs stands by top lawyer who bashed ...,https://nypost.com/2025/11/14/business/goldman...,95
68,2025-11-14-22-56-00 +0000,wsj,"Trump moved to lower duties on beef, coffee an...",https://www.wsj.com/economy/trade/trump-implem...,94


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
33,126,2025-11-15-01-02-00 +0000,wsj,Trump said he would ask the Justice Department...,https://www.wsj.com/politics/policy/trump-call...
124,43,2025-11-14-21-07-00 +0000,wsj,Trump said he is withdrawing the nomination of...,https://www.wsj.com/politics/policy/trump-with...
111,42,2025-11-14-21-47-00 +0000,wsj,The reopened federal government will on Thursd...,https://www.wsj.com/economy/jobs/september-job...
86,40,2025-11-14-22-26-25 +0000,nypost,Jack Schlossberg shamelessly invokes Kennedy n...,https://nypost.com/2025/11/14/us-news/jack-sch...
135,40,2025-11-14-20-10-03 +0000,nyt,Israeli Settler Attack During West Bank Olive ...,https://www.nytimes.com/2025/11/14/world/middl...
211,38,2025-11-14-15-36-00 +0000,wsj,Who Is John Furner? Meet the Walmart Insider N...,https://www.wsj.com/business/retail/john-furne...
299,35,2025-11-14-09-00-00 +0000,cbc,5-year-old Masa has been missing for more than...,https://www.cbc.ca/news/world/gaza-rubble-bodi...
126,35,2025-11-14-21-03-39 +0000,nypost,Purdue Pharma’s $7B opioid bankruptcy deal to ...,https://nypost.com/2025/11/14/business/judge-a...
207,34,2025-11-14-16-00-00 +0000,wsj,"Our compulsive purchasing of cheap, foreign-ma...",https://www.wsj.com/economy/consumers/will-tru...
198,30,2025-11-14-16-20-39 +0000,bbc,Reeves backs away from income tax rates rise a...,https://www.bbc.com/news/articles/c0mx3nv7zy8o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
